In [2]:
!pip install gensim

zsh:1: command not found: pip


In [4]:
import gensim.downloader as api

ModuleNotFoundError: No module named 'gensim'

In [5]:
pretrained_word2vec_model = api.load("word2vec-google-news-300")

NameError: name 'api' is not defined

In [ ]:
pretrained_glove_model = api.load("glove-wiki-gigaword-300")

In [ ]:
pretrained_fasttext_model = api.load("fasttext-wiki-news-subwords-300")

#1. Data Acquisiton

In [ ]:
import pandas as pd

In [ ]:
amazon_df  = pd.read_csv("amazon_cells_labelled.txt",sep='\t',header=None)

In [ ]:
yelp_df = pd.read_csv("yelp_labelled.txt",sep='\t', header=None)

In [ ]:
imdb_df = pd.read_csv("imdb_labelled.txt",sep='\t',header=None)

In [ ]:
#Combining all the datasets into one big dataset
master_df = pd.concat([amazon_df,yelp_df,imdb_df])

In [ ]:
master_df

In [ ]:
master_df[1].value_counts()

#2. Text Cleaning & Pre-processing

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [ ]:
def clean_preprocess(text):
  #Lowercasing
  text = text.lower()
  #Removing punctuation and digits
  doc = nlp(text)
  text = [token.text  for token in doc if (not token.is_punct) and (not token.like_num)]
  #Removing stopwords
  preprocessed_text = [word for word in text if word not in ENGLISH_STOP_WORDS]
  #preprocessed_text = ' '.join(text)#Joining list of word into a sentence
  return preprocessed_text

In [ ]:
master_df[0] = master_df[0].apply(lambda row: clean_preprocess(row))

In [ ]:
master_df

#3. Text Representation or Feature Engineering (using GloVe and fastText pretrained models)

In [ ]:
import numpy as np

In [ ]:
def convert_word_to_vector_w2v(text):
  total_vector = []
  for row in text:#Iterating each row in the text column of the dataset
    count = 0
    row_total_vector = np.zeros(300) #initializing with the zeros of the same size as vector
    for word in row:#Iterating over each word in a row
      if word in pretrained_word2vec_model:
        row_total_vector = row_total_vector + pretrained_word2vec_model[word]
        count = count + 1
    if count != 0:
      row_avg_vector = row_total_vector/count
      total_vector.append(row_avg_vector)
    else:
      total_vector.append(np.zeros(300))


  return total_vector


In [ ]:
X_vector_w2v = convert_word_to_vector_w2v(master_df[0])

In [ ]:
def convert_word_to_vector_glove(text):
  total_vector = []
  for row in text:#Iterating each row in the text column of the dataset
    count = 0
    row_total_vector = np.zeros(300) #initializing with the zeros of the same size as vector
    for word in row:#Iterating over each word in a row
      if word in pretrained_glove_model:
        row_total_vector = row_total_vector + pretrained_glove_model[word]
        count = count + 1
    if count != 0:
      row_avg_vector = row_total_vector/count
      total_vector.append(row_avg_vector)
    else:
      total_vector.append(np.zeros(300))


  return total_vector


In [ ]:
X_vector_glove = convert_word_to_vector_glove(master_df[0])

In [ ]:
def convert_word_to_vector_fasttext(text):
  total_vector = []
  for row in text:#Iterating each row in the text column of the dataset
    count = 0
    row_total_vector = np.zeros(300) #initializing with the zeros of the same size as vector
    for word in row:#Iterating over each word in a row
      if word in pretrained_fasttext_model:
        row_total_vector = row_total_vector + pretrained_fasttext_model[word]
        count = count + 1
    if count != 0:
      row_avg_vector = row_total_vector/count
      total_vector.append(row_avg_vector)
    else:
      total_vector.append(np.zeros(300))


  return total_vector


In [ ]:
X_vector_fasttext = convert_word_to_vector_fasttext(master_df[0])

#4. Training the model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_vector_w2v,master_df[1],random_state=1)

In [ ]:
model_w2v = LogisticRegression()#Defining the M.L. Algorithm
model_w2v.fit(X_train_w2v, y_train_w2v) #Training the model

In [ ]:
X_train_glove, X_test_glove, y_train_glove, y_test_glove = train_test_split(X_vector_glove,master_df[1],random_state=1)

In [ ]:
model_glove = LogisticRegression()#Defining the M.L. Algorithm
model_glove.fit(X_train_glove, y_train_glove) #Training the model

In [ ]:
X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X_vector_fasttext,master_df[1],random_state=1)

In [ ]:
model_fasttext = LogisticRegression()#Defining the M.L. Algorithm
model_fasttext.fit(X_train_ft, y_train_ft) #Training the model

#5. Evaluating the model

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
model_w2v.score(X_test_w2v,y_test_w2v)

In [ ]:
model_glove.score(X_test_glove,y_test_glove)

In [ ]:
model_fasttext.score(X_test_ft,y_test_ft)

#6. Saving the best model

In [ ]:
import pickle

with open('sentiment_model.pkl', 'wb') as file:
    pickle.dump(model_glove, file)